### Step 4
# Evaluation of completion results



In [ ]:
import sys, os
lib_dir = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/src"))
if lib_dir not in sys.path:
    print(f"Adding {lib_dir} to sys.path")
    sys.path.insert(0, lib_dir)
else:
    print(f"{lib_dir} already in sys.path")

os.environ["APP_CONFIG_DIR"] = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/config"))

data_path = os.path.abspath(os.path.join(os.path.curdir, "..", "..", "eval-data"))

import dotenv
dotenv.load_dotenv()

# Load the Responses

In [ ]:
import json

qa_source_file = 'sample-occ-uce-mc-exam.json'

qa_input_file = os.path.join(data_path, "responses", qa_source_file)

with open(qa_input_file, 'r') as f:
    file_data = f.read()
    qa_data = json.loads(file_data)

print(f"Loaded {len(qa_data.get("eval-questions"))} responses from {qa_input_file}")



# Perform evals

In [ ]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric


metric = AnswerRelevancyMetric(threshold=0.5)
test_cases = [LLMTestCase(
    input=response.get("question"),
    actual_output=response.get("response")
) for response in qa_data.get("eval-questions")]

score_data = evaluate(test_cases, [metric])
scores = [r.metrics_data[0].score for r in score_data.test_results]
print(scores)



In [7]:
for qa, s in zip(qa_data.get("eval-questions"), scores):
    qa["score-answer-relevance"] = s

output_file = os.path.join(data_path, "scores", f"{qa_source_file}.scored.json")

with open(output_file, 'w') as f:
    f.write(json.dumps(qa_data, indent=2))
    